Insert explanatory text here. Allow 1 hour for completion.

In [1]:
load("preamble.sage")

Construct the set of $\mathbb{F}_2$-rational points of $G(2,5)$, represented as $2 \times 5$ echelon matrices,
together with their Plucker coordinates.

In [2]:
F = GF(2)

In [3]:
coords = {}
for V in VectorSpace(F, 5).subspaces(2):
    M = V.matrix()
    M.set_immutable()
    coords[M] = vector(M.minors(2))
S = list(coords.keys())
len(S)

155

Construct the set of $\mathbb{F}_4$-rational points of $G(2,5)$, retaining only the Plucker coordinates.

In [4]:
F4 = GF(4)
S4 = []
for V in VectorSpace(F4, 5).subspaces(2):
    M = V.matrix()
    v = vector(M.minors(2))
    i = min(j for j in range(10) if v[j])
    assert v[i] == 1
    S4.append(v)
len(S4)

5797

Construct the Plucker relations defining $G(2,5)$ inside $\mathbb{P}^9$, then verify that these relations vanish on the points we have constructed.

In [5]:
P.<x01,x02,x03,x04,x12,x13,x14,x23,x24,x34> = PolynomialRing(F, 10)
quads = [x01*x23 + x02*x13 + x03*x12,
         x01*x24 + x02*x14 + x04*x12,
         x01*x34 + x03*x14 + x04*x13,
         x02*x34 + x03*x24 + x04*x23,
         x12*x34 + x13*x24 + x14*x23]
assert all(f(*v) == 0 for f in quads for v in coords.values())

Construct an orbit lookup tree of depth 6. We forbid 5-tuples whose linear span is only 3-dimensional, as these can only occur for plane quintics.

In [6]:
def vecs_to_gens(vecs):
    return [sum(P.gens()[i] * v[i] for i in range(10)) for v in vecs]

In [7]:
def linear_section(coords):
    V = Matrix(coords).right_kernel()
    tmp2 = vecs_to_gens(V.gens()) + quads
    return P.ideal(tmp2)

In [8]:
def apply_group_elem(g, M, set_immutable=True):
    M1 = (M*~g).echelon_form()
    if set_immutable:
        M1.set_immutable()
    return M1

In [9]:
def stabilizer(M):
    # First construct the stabilizer of the span of e1, e2.
    l0 = [block_matrix(2,2,[g.matrix(),0,0,identity_matrix(3)], subdivide=False) for g in GL(2, F).gens()] + \
        [block_matrix(2,2,[identity_matrix(2),0,0,g.matrix()], subdivide=False) for g in GL(3, F).gens()]
    l0.append(identity_matrix(5))
    l0[-1][3,0] = 1
    # Construct a matrix to conjugate these generators.
    l = M.rows()
    for i in range(5):
        if i not in M.pivots():
            l.append(vector(1 if j==i else 0 for j in range(5)))
    M1 = Matrix(F, l)
    G = GL(5, F).subgroup([~M1*g*M1 for g in l0])
    assert all((M*g.matrix()).echelon_form() == M for g in G.gens())
    return G

In [10]:
def optimized_rep(g):
    return g.matrix()

In [11]:
def forbid(mats, easy=False, coords=coords):
    tmp = [coords[M] for M in mats]
    if len(mats) == 5 and Matrix(tmp).rank() <= 3:
        return True
    J = linear_section(tmp)
    if J.dimension() > 3:
        return True
    if easy:
        return False
    return False

In [12]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': vec_stab,
           'optimized_rep': optimized_rep,
           'forbid': forbid}

In [13]:
G = GL(5, F)
tree = build_orbit_tree(G, S, 6, methods, verbose=False)

In [14]:
[len(green_nodes(tree, i)) for i in range(4,7)]

[39, 235, 2201]

Construct the linear spans of these tuples.

In [15]:
d = defaultdict(list)
for i in range(4, 7):
    for mats in green_nodes(tree, i):
        tmp = [coords[M] for M in mats]
        m = Matrix(tmp).echelon_form()
        W = m.row_space()
        d[mats].append(W)

Construct 6-dimensional extensions of these subspaces (in terms of bases thereof).

In [16]:
d2 = defaultdict(list)
V0 = VectorSpace(F, 10)
for mats in d:
    V1 = V0.subspace([coords[M] for M in mats])
    for V2 in subspaces_containing(V0, V1, 6-V1.dimension()):
        J = linear_section(V2.gens())
        if J.dimension() > 3:
            continue
        d2[mats].append(V2)

In [17]:
sum(len(d2[mats]) for mats in d2)

151917

For each 6-dimensional subspace, identify quadrics which meet this subspace in exactly the specified set of $\mathbb{F}_2$-rational points, modulo multiples of the linear relations cutting out the subspaces and the quadrics cutting out the Grassmannian. Then count $\mathbb{F}_4$-points and retain cases fitting our targets.

In [18]:
monos2 = [prod(x) for x in itertools.combinations_with_replacement(P.gens(), 2)]
len(monos2)

55

In [19]:
coords2 = {x: vector(F, (mu(*coords[x]) for mu in monos2)) for x in S}

In [20]:
def redundancy(gens, F=F, P=P, monos2=monos2):
    return [vector(F, ((gen*y).coefficient(mu) for mu in monos2)) for gen in gens for y in P.gens()] + \
       [vector(F, (gen.coefficient(mu) for mu in monos2)) for gen in quads]

In [21]:
curves = defaultdict(list)
tmp2 = [t[:2] for t in targets6]
for mats in d2:
    s = len(mats)
    for V in d[mats]:
        V1 = V.matrix().right_kernel()
        gens = vecs_to_gens(V1.basis())
        pts = [x for x in S if coords[x] in V]
        pts2 = 0
        perp = Matrix([coords2[x] for x in pts])
        target = vector(F, (0 if x in mats else 1 for x in pts))
        for w in solve_right_iterator(perp, target, redundancy, gens):
            if pts2 == 0:
                pts2 = [x for x in S4 if all(gen(*x) == 0 for gen in gens)]
            gens1 = sum(w[i]*monos2[i] for i in range(55))
            s2 = sum(1 for x in pts2 if gens1(*x) == 0)
            if (s,s2) in tmp2:
                curves[(s,s2)].append(gens + [gens1])

In [22]:
[(s, len(curves[s])) for s in curves]

[((5, 13), 17), ((5, 11), 21), ((5, 15), 9), ((6, 14), 1600), ((6, 10), 5172)]

Compute point counts over $\mathbb{F}_{q^i}$ for $i=3,4$, retaining cases that fit our targets.

In [23]:
def count_by_ideal(gens, n):
    J = P.ideal(gens + quads + [y^(2^n) + y for y in P.gens()])
    return (J.vector_space_dimension() - 1) // (2^n-1)

In [24]:
for n in range(3, 5):
    tmp = [t[:n] for t in targets6]
    tmp2 = list(curves.keys())    
    for s in tmp2:
        for gens in curves[s]:
            i = count_by_ideal(gens, n)
            s1 = s + (i,)
            if s1 in tmp:
                curves[s1].append(gens)
        del curves[s]
    print([(s, len(curves[s])) for s in curves])

[((5, 11, 11), 17), ((5, 15, 5), 2), ((6, 14, 12), 255), ((6, 14, 6), 121), ((6, 10, 9), 791)]
[((5, 11, 11, 31), 4), ((6, 14, 12, 26), 5), ((6, 14, 6, 26), 7), ((6, 14, 6, 34), 3), ((6, 10, 9, 38), 10)]


Use Magma to compute the full zeta functions of the resulting curves (discarding cases that do not lead to smooth curves of the right genus), and compare these to our targets.

In [25]:
proj_magma = magma.ProjectiveSpace(P)
l = []
for s in curves:
    for gens in curves[s]:
        X = proj_magma.Scheme(gens + quads)
        if X.Dimension() > 1 or str(X.IsIrreducible()) == "false":
            continue
        C = X.Curve()
        if C.Genus() != 6:
            continue
        ct = tuple(Integer(C.NumberOfPlacesOfDegreeOneECF(i)) for i in range(1, 7))
        if ct in targets6:
            l.append(C)
len(l)

6

In [26]:
l = isomorphism_class_reps(l)
len(l)

4

Search for extensions of relative class number 1.

In [27]:
l2 = []
for C in l:
    F0 = magma.FunctionField(C)
    F1 = F0.AlgorithmicFunctionField()
    l2 = l2 + match_weil_poly(F1, 2)

In [28]:
len(l2)

1

In [34]:
for (F1, F2) in l2:
    t = tuple(F1.NumberOfPlacesOfDegreeOneECF(i) for i in range(1,7))
    print(t, F1.RationalExtensionRepresentation())

(6, 14, 12, 26, 6, 56) Algebraic function field defined over Univariate rational function field over GF(2) by
($.1^2 + $.1 + 1)*$.1^4 + ($.1^3 + $.1 + 1)*$.1^3 + ($.1^4 + $.1^2 + 1)*$.1^2 + ($.1^5 + $.1^4 + 1)*$.1 + $.1^5 + $.1^4 + $.1^3 + $.1


Convert the covers into numerical data in preparation for writing out to the spreadsheet.

In [30]:
l3 = []
Q.<T> = QQ[]
for (F, F1) in l2:
    u1 = T.parent()(magma_poly_list(F.ZetaFunction().Numerator())).reverse()
    v1 = T.parent()(magma_poly_list(F1.ZetaFunction().Numerator())).reverse()
    l3.append((2, 6, 11, point_count_from_weil_poly(u1, 13), point_count_from_weil_poly(v1, 13)))

Write the results back to the spreadsheet. In the process, we check that every covering we found gives a pair of Weil polynomials from our original list.

In [33]:
used_pols = []
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    ct1 = eval(r["Counts of C"])
    ct2 = eval(r["Counts of C'"])
    if (d, g, g1, ct1, ct2) in l3:
        df.loc[i, "Cyclic"] = "Yes"
        used_pols.append((d, g, g1, ct1, ct2))
assert all(t in used_pols for t in l3)

In [32]:
df.to_excel('../Shared/polys.xlsx', sheet_name='Weil polynomials', merge_cells=True, freeze_panes=(int(1),int(1)))